In [50]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras import models, layers, activations, initializers, losses, optimizers, metrics
from keras.models import Sequential, Model
from keras.layers import Dense, Input, LeakyReLU, Dropout, BatchNormalization
from keras.layers.merge import concatenate
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [51]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    length = len(sequence)
    col_length = len(sequence[0])
    for i in range(length):
        end_ix = i + n_steps
        if end_ix > length-1:
            break
        seq_x, seq_y = sequence[i:end_ix, :], sequence[end_ix, col_length-1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [52]:
kospi = pd.read_csv('data/kospi200.csv', encoding='euc-kr')
kospi = kospi.sort_values(['일자'], ascending=[True])
kospi = kospi.iloc[:,1:].copy()

kospi = pd.concat([kospi.iloc[:,:3], kospi.iloc[:,4], kospi.iloc[:,3]], axis=1)

kospi['거래량'] = kospi['거래량'].apply(lambda x: x.replace(",","",-1))

for i in kospi.columns:
    kospi[i] = kospi[i].astype(float)

kospi = np.array(kospi)

In [53]:
data = pd.read_csv('data/samsung.csv', encoding='euc-kr')
data = data.sort_values(['일자'], ascending=[True])
data = data.iloc[:,1:].copy()

data = pd.concat([data.iloc[:,:3], data.iloc[:,4], data.iloc[:,3]], axis=1)

for i in data.columns:
    data[i] = data[i].apply(lambda x: x.replace(",","",-1))

for i in data.columns:
    data[i] = data[i].astype(int)

data = np.array(data)

In [54]:
n_steps = 5
samsung_x, samsung_y = split_sequence(data, n_steps)
kospi_x, _ = split_sequence(kospi, n_steps)

In [55]:
samsung_x = samsung_x.reshape((samsung_x.shape[0], -1))
scaler = StandardScaler()
scaler.fit(samsung_x)
samsung_x = scaler.transform(samsung_x)
samsung_x = samsung_x.reshape((samsung_x.shape[0], -1, 1))
print(samsung_x.shape)
kospi_x = kospi_x.reshape((kospi_x.shape[0], -1))
scaler = StandardScaler()
scaler.fit(kospi_x)
kospi_x = scaler.transform(kospi_x)
kospi_x = kospi_x.reshape((kospi_x.shape[0], -1, 1))
print(kospi_x.shape)

(421, 25, 1)
(421, 25, 1)


In [56]:
final_predict_samsung, final_predict_kospi = samsung_x[-1:,:], kospi_x[-1:,:]

In [57]:
from sklearn.model_selection import train_test_split

samsung_x_train, samsung_x_test, kospi_x_train, kospi_x_test, y_train, y_test = train_test_split(
    samsung_x, kospi_x, samsung_y, train_size=0.9, random_state=66, shuffle=True
)

In [58]:
inp1 = Input(shape=(samsung_x.shape[1],samsung_x.shape[2]))
x = LSTM(128, activation='relu')(inp1)
x = Dense(256, kernel_initializer='he_normal')(x)
x = LeakyReLU(alpha=0.05)(x)
x = Dropout(0.25)(x)
x = Dense(128, kernel_initializer='he_normal')(x)
x = LeakyReLU(alpha=0.05)(x)
x = Dropout(0.25)(x)
x = Dense(64, kernel_initializer='he_normal')(x)
out1 = LeakyReLU(alpha=0.05)(x)

inp2 = Input(shape=(kospi_x.shape[1],kospi_x.shape[2]))
x1 = LSTM(128, activation='relu')(inp2)
x1 = Dense(256, kernel_initializer='he_normal')(x1)
x1 = LeakyReLU(alpha=0.05)(x1)
x1 = Dropout(0.4)(x1)
x1 = Dense(128, kernel_initializer='he_normal')(x1)
x1 = LeakyReLU(alpha=0.05)(x1)
x1 = Dropout(0.25)(x1)
x1 = Dense(64, kernel_initializer='he_normal')(x1)
out2 = LeakyReLU(alpha=0.05)(x1)

merge = concatenate([out1, out2])
merge = Dense(32, activation='relu', kernel_initializer='he_normal')(merge)
merge = Dense(16, activation='relu', kernel_initializer='he_normal')(merge)
output = Dense(1, activation='linear', kernel_initializer='he_normal')(merge)

model = Model(inputs=[inp1, inp2], outputs=output)

In [59]:
model.compile(loss='mse', optimizer='adam', metrics=['acc'])
model.fit([samsung_x_train, kospi_x_train], y_train, epochs=100, batch_size=1)

Epoch 1/100
378/378 [==============================] - 9s 24ms/step - loss: 7100712551.3704 - acc: 0.0000e+00
Epoch 2/100
378/378 [==============================] - 6s 16ms/step - loss: 1993359237.5026 - acc: 0.0000e+00
Epoch 3/100
378/378 [==============================] - 6s 16ms/step - loss: 465414941293.6411 - acc: 0.0000e+00
Epoch 4/100
378/378 [==============================] - 6s 16ms/step - loss: 478737438.9299 - acc: 0.0000e+00
Epoch 5/100
378/378 [==============================] - 6s 16ms/step - loss: 180256816.2693 - acc: 0.0000e+00: 0s - loss: 184445816.7899 - acc
Epoch 6/100
378/378 [==============================] - 6s 16ms/step - loss: 100152969.4462 - acc: 0.0000e+00
Epoch 7/100
378/378 [==============================] - 6s 16ms/step - loss: 72371936.1624 - acc: 0.0000e+00
Epoch 8/100
378/378 [==============================] - 6s 16ms/step - loss: 70464542.5961 - acc: 0.0000e+00
Epoch 9/100
378/378 [==============================] - 6s 16ms/step - loss: 54544534.3612 - 

378/378 [==============================] - 5s 13ms/step - loss: 8950598.9141 - acc: 0.0000e+00
Epoch 76/100
378/378 [==============================] - 5s 13ms/step - loss: 10116543.3172 - acc: 0.0000e+00
Epoch 77/100
378/378 [==============================] - 5s 13ms/step - loss: 8356709.5223 - acc: 0.0000e+00
Epoch 78/100
378/378 [==============================] - 5s 13ms/step - loss: 9277285.5642 - acc: 0.0000e+00
Epoch 79/100
378/378 [==============================] - 5s 13ms/step - loss: 8479509.2111 - acc: 0.0000e+00
Epoch 80/100
378/378 [==============================] - 5s 13ms/step - loss: 9476193.5334 - acc: 0.0000e+00
Epoch 81/100
378/378 [==============================] - 5s 13ms/step - loss: 7886427.6564 - acc: 0.0026
Epoch 82/100
378/378 [==============================] - 5s 13ms/step - loss: 8170773.2556 - acc: 0.0000e+00
Epoch 83/100
378/378 [==============================] - 5s 13ms/step - loss: 8542709.7002 - acc: 0.0000e+00
Epoch 84/100
378/378 [======================

In [63]:
y_predict = model.predict([samsung_x_test, kospi_x_test])

In [64]:
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict): # 실제 정답값, 모델을 통한 예측값 
    return np.sqrt(mean_squared_error(y_test, y_predict))
print('RMSE :', RMSE(y_test, y_predict))

RMSE : 1192.030703324833


In [65]:
print(model.predict([final_predict_samsung, final_predict_kospi]))

[[56261.637]]
